# 

In [1]:
# Импортируем библиотеки
import torch
import warnings
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import (
                          GPT2Tokenizer, LineByLineTextDataset, DataCollatorForLanguageModeling,
                          Trainer, TrainingArguments, AutoModelForCausalLM
                         )


# Игнорируем предупреждения
warnings.filterwarnings('ignore')

# Освободим всю незанятую кэшированную память
torch.cuda.empty_cache()

# Проверяем, работает ли cuda
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

# Опеределяем путь до Dataset
path_to_folder = 'Dataset.xlsx'

cuda


In [12]:
# Загружаем Dataset и оставляем нужные колонки
data = pd.read_excel(path_to_folder).dropna()[['Условие', 'Решение']]
data.head(3)

,Условие,Решение
0,"Разложите на простые множители числа 111, 1111...","Разложим на простые множители числа 111, 1111,..."
1,"Изменятся ли частное и остаток, если делимое и...","Частное не изменится, а остаток увеличится втр..."
2,Можно ли доску размером 5×5 заполнить доминошк...,Общее количество клеток [math]calk_evalWithNum...


In [13]:
# Разделение датасета на обучающий и валидационный
train_data, val_data = train_test_split(data, test_size=0.1, random_state=42)

# Инициализация токенизатора
tokenizer = GPT2Tokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")
model = AutoModelForCausalLM.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

# Создание списка строк, объединяя данные из колонок "Условие" и "Решение"
train_texts = train_data['Условие'] + ' ' + train_data['Решение']
val_texts = val_data['Условие'] + ' ' + val_data['Решение']

# Запись данных в текстовые файлы
with open('train_dataset.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(train_texts))
    
with open('val_dataset.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(val_texts))

# Создание датасетов для обучения и валидации
train_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path='train_dataset.txt',
    block_size=2000,
)

val_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path='val_dataset.txt',
    block_size=2000,
)

# Коллатор данных
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

# Настройка параметров обучения
training_args = TrainingArguments(
    output_dir="./finetuned_rugpt",  # Директория для сохранения обученной модели
    overwrite_output_dir=True,
    num_train_epochs=20,  # Количество эпох обучения
    per_device_train_batch_size=8,  # Размер пакета обучения
    save_steps=10_000,  # Как часто сохранять модель
    save_total_limit=3,  # Ограничение на количество сохраненных моделей
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

In [14]:
# Обучение модели
trainer.train()

# Сохранение обученной модели
trainer.save_model("./finetuned_rugpt")

Step,Training Loss
500,2.329600


# Функция получения предсказания

In [9]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = GPT2LMHeadModel.from_pretrained("./finetuned_rugpt").to(device)
tokenizer = GPT2Tokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

def generate_response(input_text, max_length=300, temperature=0.7):
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)
    output = model.generate(
        input_ids,
        max_length=max_length,
        temperature=temperature,
        pad_token_id=tokenizer.eos_token_id,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        top_k=5,
        top_p=0.95,
        eos_token_id=tokenizer.eos_token_id,
        early_stopping=True
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)

input_text = "В футбольной команде (11 человек) нужно выбрать капитана и его заместителя. Сколькими способами это можно сделать?"
response = generate_response(input_text)
print("Ответ:", response[len(input_text):])

Ответ:  Выбор капитана должен быть сделан в пользу его заместителей. Выбор первого заместителя не имеет значения, поскольку он не может быть выбран ни при каких обстоятельствах. Заместители могут выбрать двух человек, но не более. Поэтому выбор первого зама не важен. В качестве первого зам. первого помощника можно выбрать одного из двух оставшихся двух. Выбираем второго из оставшихся трех человек. При этом второй из них может выбрать первого, а второй – второго. Таким образом, выбор второго и выбор третьего должен сочетаться. Однако при этом оба человека не могут одновременно выбрать второго, так как оба они не смогут выбрать ни одного капитана. Следовательно, второй и третий должны выбрать третьего. Итак, всего способов выбрать трех заместительных. Первый вариант. Второй вариант выбора второго заместителя – это выбор капитана, который не зависит от выбора первого. Но при таком выборе второго капитана выбор будет равняться 2. Значит, второго – 3. А второго (в данном случае второго) – 4